# Extend `h5io` interface
In addition to the `Pointer()` class, the `h5io_browser` package also extends the interface of `h5io` with the following four functions: 
* `read_dict_from_hdf()` to read an hierarchical dictionary from the HDF5 file.
* `write_dict_to_hdf()` to write an hierarchical dictionrary from the HDF5 file.
* `list_hdf()` to list the content of an HDF5 file.
* `delete_item()` to delete an item in a given HDF5 file.

A hierarchical python dictionary can be written in two way, either as a python dictionary with `h5io` automatically determining which groups and nodes are going to be created or by the user specifying the hierarchy of the objects in the dictionary. The second approach allows the user to reload individual objects from the HDF5 file later on, which is one of the key advantages of using a hierarchical data fromat like HDF5.  

In [1]:
import os
import numpy as np
from h5io_browser import read_dict_from_hdf, write_dict_to_hdf, list_hdf, delete_item

# Write dictionary
In this case `h5io` automatically determines which groups and nodes are created. More specifially, only the keys in the root level of the dictionary are resolved.

In [2]:
data = {"a": [1, 2], "b": 3, "c": {"d": 4, "e": np.array([5, 6])}}

In [3]:
file_name = "test_1.h5"
if os.path.exists(file_name):
    os.remove(file_name)

In [4]:
write_dict_to_hdf(file_name=file_name, data_dict=data)

The content of this HDF5 file is always represented as the list of keys of the root level of the dictionary. 

In [5]:
list_hdf(file_name=file_name, h5_path="/")

(['/a', '/b', '/c'], [])

Consequently, it makes no difference if `read_dict_from_hdf()` is called with `recursive=True` or `recursive=False`:

In [6]:
read_dict_from_hdf(file_name=file_name, h5_path="/")

{'a': [1, 2], 'b': 3, 'c': {'d': 4, 'e': array([5, 6])}}

In [7]:
read_dict_from_hdf(file_name=file_name, h5_path="/", recursive=True)

{'a': [1, 2], 'b': 3, 'c': {'d': 4, 'e': array([5, 6])}}

## Delete data 
In the same way keys from the root dictionary can be deleted using the `delete_item()` function:

In [8]:
delete_item(file_name=file_name, h5_path="a")

In [9]:
list_hdf(file_name=file_name, h5_path="/")

(['/b', '/c'], [])

In [10]:
read_dict_from_hdf(file_name=file_name, h5_path="/")

{'b': 3, 'c': {'d': 4, 'e': array([5, 6])}}

# Write hierarchical dictionary
The alternative approach is for the user to specify which keys should be resolved. This can be done, by reducing the dictionary to a single level, namely convert the hierarchical dictionary from above in a flat dictionary with the keys providing the path inside the hierarchy.

In [11]:
data_structured = {
    "a": data["a"],
    "b": data["b"],
    "c/d": data["c"]["d"],
    "c/e": data["c"]["e"],
}

In [12]:
file_name = "test_2.h5"
if os.path.exists(file_name):
    os.remove(file_name)

In [13]:
write_dict_to_hdf(file_name=file_name, data_dict=data_structured)

## Read data 
In this case the hierarchical part of the dictionary, in this case the key `c` is represented as an HDF5 group. In this way the nodes inside the HDF5 group can be read individually:

In [14]:
list_hdf(file_name=file_name, h5_path="/")

(['/a', '/b'], ['/c'])

In [15]:
list_hdf(file_name=file_name, h5_path="/c")

(['/c/d', '/c/e'], [])

In [16]:
read_dict_from_hdf(file_name=file_name, h5_path="/")

{'a': [1, 2], 'b': 3}

In [17]:
read_dict_from_hdf(file_name=file_name, h5_path="/", recursive=True)

{'a': [1, 2], 'b': 3, 'c': {'d': 4, 'e': array([5, 6])}}

## Delete data
Deleting data works just like before:

In [18]:
delete_item(file_name=file_name, h5_path="a")

In [19]:
list_hdf(file_name=file_name, h5_path="/")

(['/b'], ['/c'])

In [20]:
read_dict_from_hdf(file_name=file_name, h5_path="/")

{'b': 3}

In [21]:
read_dict_from_hdf(file_name=file_name, h5_path="/", recursive=True)

{'b': 3, 'c': {'d': 4, 'e': array([5, 6])}}